In [ ]:
from pathlib import Path
import shutil
import zipfile

# Kaggle の入力データセット slug（username/slug の slug 部分）をセット
DATASET_SLUG = 'gns-codes'  # 変更する場合はここを更新
DATASET_ROOT = Path(f'/kaggle/input/{DATASET_SLUG}')
WORK_ROOT = Path('/kaggle/working')
repo_dir = WORK_ROOT / 'code'

code_dir = DATASET_ROOT / 'code'
code_zip = DATASET_ROOT / 'code.zip'

# 取得優先順位: 展開済み code/ があればそれを使う。無ければ code.zip を展開。どちらも無ければエラー。
if code_dir.exists():
    src = code_dir
elif code_zip.exists():
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    with zipfile.ZipFile(code_zip) as zf:
        zf.extractall(repo_dir)
    src = repo_dir
else:
    raise FileNotFoundError(f"/kaggle/input/{DATASET_SLUG} に code ディレクトリも code.zip も見つかりません。Add Data で {DATASET_SLUG} を追加してください。")

if src != repo_dir:
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    shutil.copytree(src, repo_dir)
# unzip でサブディレクトリ1階層挟まった場合に平坦化
if repo_dir.exists():
    children = list(repo_dir.iterdir())
    if len(children)==1 and children[0].is_dir():
        inner = children[0]
        for path in inner.iterdir():
            path.rename(repo_dir / path.name)
        inner.rmdir()
%cd $repo_dir
# 依存パッケージは Kaggle 環境の標準ライブラリを利用（追加インストールなし）


In [ ]:
!pip install torch-geometric 
# torch-scatter torch-sparse torch-cluster

In [ ]:
import torch

torch_ver = torch.__version__.split("+")[0]          # 例: "2.1.0+cu121" -> "2.1.0"
cuda_ver  = torch.version.cuda                      # 例: "12.1" / CPUなら None
cuda_tag  = "cpu" if cuda_ver is None else "cu" + cuda_ver.replace(".", "")

url = f"https://data.pyg.org/whl/torch-{torch_ver}+{cuda_tag}.html"
print("PyG wheels:", url)

# 必要なやつだけ入れる（radius_graphなら torch_cluster が本体）
!pip -q install torch-cluster -f {url}

# もし他も使うならまとめて
# !pip -q install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f {url}
# !pip -q install torch_geometric


In [ ]:
from pathlib import Path
import yaml

REPO = Path('/kaggle/working/code')
cfg_path = REPO / 'config.yaml'

with cfg_path.open('r', encoding='utf-8') as f:
    cfg = yaml.safe_load(f)

cfg['method'] = 'hamiltonian_sph'
cfg['amp_enable'] = False
cfg['ntraining_steps'] = 10000
cfg['train_dataset_count'] = 100
cfg['valid_dataset_count'] = 20
cfg['validation_interval'] = 1000
cfg['max_grad_norm'] = 100

# Kaggle 入力データセットのパスに合わせて上書き
dataset_root = '/kaggle/input/dam-break-left-wall2'
cfg['data_path'] = dataset_root
cfg.setdefault('scenario_options', {}).setdefault('fluid', {})['dataset'] = dataset_root

with cfg_path.open('w', encoding='utf-8') as f:
    yaml.safe_dump(cfg, f, allow_unicode=True)


In [ ]:
%cd /kaggle/working/code

import subprocess
import torch

gpu_count = torch.cuda.device_count()
if gpu_count >= 2:
    nproc = 2
    cmd = ["torchrun", f"--nproc_per_node={nproc}", "src/train.py", "--config", "config.yaml"]
elif gpu_count == 1:
    nproc = 1
    cmd = ["python", "src/train.py", "--config", "config.yaml"]
else:
    nproc = 0
    cmd = ["python", "src/train.py", "--config", "config.yaml"]
    print("GPU が検出できませんでした。CPU で実行します。")

print(f"検出GPU数: {gpu_count} -> 実行コマンド: {' '.join(cmd)}")
subprocess.run(cmd, check=True)


In [ ]:
from pathlib import Path
import yaml

REPO = Path('/kaggle/working/code')
cfg_path = REPO / 'config_rollout.yaml'

with cfg_path.open('r', encoding='utf-8') as f:
    cfg = yaml.safe_load(f)

cfg['method'] = 'hamiltonian_sph'
cfg['rollout_inference_max_examples'] = 1

# 推論時のデータセットパスを上書き
dataset_root = '/kaggle/input/dam-break-left-wall2'
cfg.setdefault('scenario_options', {}).setdefault('fluid', {})['dataset'] = dataset_root

with cfg_path.open('w', encoding='utf-8') as f:
    yaml.safe_dump(cfg, f, allow_unicode=True)


In [ ]:
%cd /kaggle/working/code
!python src/train.py --config config_rollout.yaml
!python analyze_rollouts.py
!python visualize_rollout.py
